In [1]:
import os

CUR_DIR = os.getcwd()
print("dir_name ", CUR_DIR)

# get the files from this directory

files_and_folders = os.listdir(CUR_DIR)
print("files: ", files_and_folders)


dir_name  /mnt/c/Repositories/School/Semester3/Research_LogicKeyLocking/cadence
files:  ['BenchParser.py', 'ITC99_KeySize=16__5bits', 'ITC99_KeySize=4__3bits', 'ITC99_KeySize=InputSize__RANEKC2', 'Ori', 't.ipynb', '__pycache__']


In [4]:
folders_root = [f for f in files_and_folders if os.path.isdir(f)]
folders_root 

['ITC99_KeySize=16__5bits',
 'ITC99_KeySize=4__3bits',
 'ITC99_KeySize=InputSize__RANEKC2',
 'Ori',
 '__pycache__']

In [45]:
# traverse through every folder and get the verilog files
import re
from typing import Literal



class VerilogFile:
    @staticmethod
    def read_file_lines(file_name):
        with open(file_name, "r") as file:
            return file.readlines()

    def __get_by_name(self, name: Literal["input", "output"], file_name: str, seen:int=0) -> int:
        lines = VerilogFile.read_file_lines(file_name)
        inputs = []
        # print('File name = ', file_name)
        for i, line in enumerate(lines):
            if f" {name} " in line:  # find the first line with ;
                if seen > 0:
                    seen -= 1
                    continue
                for j in range(i, len(lines)):
                    cur_inputs = re.split(",| ", lines[j])  # split by comma or space
                    cur_inputs = [
                        i.replace(";", "").replace("'", "").replace(")", "").strip()
                        for i in cur_inputs
                        if i.strip()
                    ]
                    inputs.extend(cur_inputs)
                    # print("cur_inputs = ", cur_inputs)
                    if ";" in lines[j]:
                        return len(inputs) - 1
        raise ValueError("No inputs found in file")

    @staticmethod
    def get_number_inputs(file_name:str, seen:int=0 ) -> int:
        return VerilogFile().__get_by_name("input", file_name, seen) + seen 

    @staticmethod
    def get_number_outputs(file_name) -> int:
        return VerilogFile().__get_by_name("output", file_name)

    @staticmethod
    def get_io(file_name) -> int:
        return VerilogFile.get_number_inputs(
            file_name, seen=1
        ) + VerilogFile.get_number_outputs(file_name)


# VerilogFile.get_io(CUR_DIR + "\\" + folders_root[0] + "\\" + "c1355_encrypted.v")

In [46]:
os.listdir(CUR_DIR + "/" + folders_root[0])

['.rs_amin.rezaei@csulb.edu.tstamp',
 'b01_encrypted',
 'b01_encrypted.v',
 'b02_encrypted',
 'b02_encrypted.v',
 'b03_encrypted',
 'b03_encrypted.v',
 'b04_encrypted',
 'b04_encrypted.v',
 'b05_encrypted',
 'b05_encrypted.v',
 'b06_encrypted',
 'b06_encrypted.v',
 'b07_encrypted',
 'b07_encrypted.v',
 'b08_encrypted',
 'b08_encrypted.v',
 'b09_encrypted',
 'b09_encrypted.v',
 'b10_encrypted',
 'b10_encrypted.v',
 'b11_encrypted',
 'b11_encrypted.v',
 'b12_encrypted',
 'b12_encrypted.v',
 'b14_encrypted',
 'b14_encrypted.v',
 'b15_encrypted',
 'b15_encrypted.v',
 'b17_encrypted',
 'b17_encrypted.v',
 'b18_encrypted',
 'b18_encrypted.v',
 'b19_encrypted',
 'b19_encrypted.v',
 'b20_encrypted',
 'b20_encrypted.v',
 'b21_encrypted',
 'b21_encrypted.v',
 'b22_encrypted',
 'b22_encrypted.v',
 'fv',
 'genus.cmd',
 'genus.cmd1',
 'genus.cmd10',
 'genus.cmd11',
 'genus.cmd12',
 'genus.cmd13',
 'genus.cmd14',
 'genus.cmd15',
 'genus.cmd16',
 'genus.cmd17',
 'genus.cmd18',
 'genus.cmd19',
 'genus

In [47]:
# Get the IO of every file in every folder
import pandas as pd 
def get_io_folder(folder):
    files = os.listdir(CUR_DIR + "/" + folder)
    print('files = ', files)
    return [{'Benchmark_name': file, 'io' :VerilogFile.get_io(CUR_DIR + "/" + folder + "/" + file)} for file in files if file.endswith('.v')]

df = pd.DataFrame( get_io_folder(folders_root[0]))


for folder in folders_root:
    print('folder = ', folder)
    print(pd.DataFrame( get_io_folder(folder) ))
    print('\n\n')

files =  ['.rs_amin.rezaei@csulb.edu.tstamp', 'b01_encrypted', 'b01_encrypted.v', 'b02_encrypted', 'b02_encrypted.v', 'b03_encrypted', 'b03_encrypted.v', 'b04_encrypted', 'b04_encrypted.v', 'b05_encrypted', 'b05_encrypted.v', 'b06_encrypted', 'b06_encrypted.v', 'b07_encrypted', 'b07_encrypted.v', 'b08_encrypted', 'b08_encrypted.v', 'b09_encrypted', 'b09_encrypted.v', 'b10_encrypted', 'b10_encrypted.v', 'b11_encrypted', 'b11_encrypted.v', 'b12_encrypted', 'b12_encrypted.v', 'b14_encrypted', 'b14_encrypted.v', 'b15_encrypted', 'b15_encrypted.v', 'b17_encrypted', 'b17_encrypted.v', 'b18_encrypted', 'b18_encrypted.v', 'b19_encrypted', 'b19_encrypted.v', 'b20_encrypted', 'b20_encrypted.v', 'b21_encrypted', 'b21_encrypted.v', 'b22_encrypted', 'b22_encrypted.v', 'fv', 'genus.cmd', 'genus.cmd1', 'genus.cmd10', 'genus.cmd11', 'genus.cmd12', 'genus.cmd13', 'genus.cmd14', 'genus.cmd15', 'genus.cmd16', 'genus.cmd17', 'genus.cmd18', 'genus.cmd19', 'genus.cmd2', 'genus.cmd3', 'genus.cmd4', 'genus.cm

In [17]:
from typing import Dict, List
from pprint import pprint
import pandas as pd

NAME_IN = 0 
LINE_IN = 1 
WORD_NUMBER = 2 
report_types :list[tuple[str, str, int]]= [  
                                        ('area', '<none> (D)', 4), 
                                        ('power', 'Subtotal', 4 )
                ] 

def get_files_in_folder(folder_name: str):
    folders = os.listdir(  folder_name)
    result : Dict[str, List[str]] = {"Benchmark_Name": [], "area": [], "power": []} 

    for f in folders: 
        dir_path = os.path.join(CUR_DIR, folder_name, f)
        # print(f'f={f} ')
        if not os.path.isdir(dir_path):
            continue
        for file in os.listdir(dir_path):
            for report, line_in, word_number in report_types:
                if report in file:
                    # print(f'\n\nf={f} ' )
                    # print(f'file = {file}')
                    if f not in result["Benchmark_Name"]:
                        result["Benchmark_Name"].append(f)
                    # check if the word is in the file, the get the value at the word number 
                    with open(os.path.join(dir_path, file), 'r') as f_:
                        lines = f_.readlines()
                        for line in lines:
                            if line_in in line:
                                # print('line = ', line)
                                words = line.split()
                                # print('words = ', words)
                                # print('value = ', words[word_number])
                                result[report].append(words[word_number])
    # pprint(result)
    return result


df = pd.DataFrame(get_files_in_folder(folders_root[0]))


In [18]:
for folder in folders_root:
    print('  \n')
    print(f'folder = {folder}')
    df = pd.DataFrame(  get_files_in_folder(folder) ) 
    print(df)

  

folder = ITC99_KeySize=16__5bits
   Benchmark_Name       area        power
0   b01_encrypted    162.792  1.55190e-06
1   b02_encrypted    175.446  1.42002e-06
2   b03_encrypted    409.374  3.65379e-06
3   b04_encrypted    920.664  9.08184e-06
4   b06_encrypted    196.308  1.66148e-06
5   b07_encrypted    638.856  5.20789e-06
6   b08_encrypted    382.356  3.03471e-06
7   b09_encrypted    381.330  3.09005e-06
8   b10_encrypted    380.988  3.12550e-06
9   b11_encrypted    699.390  5.73569e-06
10  b12_encrypted   1633.392  1.16794e-05
11  b14_encrypted   4508.928  5.22304e-05
12  b15_encrypted   7653.276  6.13533e-05
13  b17_encrypted  25205.810  2.00410e-04
14  b18_encrypted  64857.906  1.96843e-03
15  b20_encrypted   9547.272  1.15532e-04
16  b21_encrypted   9598.572  1.18406e-04
17  b22_encrypted  13952.916  1.68472e-04
  

folder = ITC99_KeySize=4__3bits
   Benchmark_Name       area        power
0   b01_encrypted    100.548  1.03275e-06
1   b02_encrypted     75.924  7.07776e-07
2  